In [1]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv("Data/joined-weather-solar.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.month
df = df.drop(columns=['Date','Station pressure', 'Unnamed: 0', 'Altimeter'])
df = df.rename(columns={'Temperature':'temp',
                        'Dew point':'dew',
                        'Wind speed':'windspeed',
                        'Cloud coverage': 'cloudcover',
                        'Visibility': 'visibility',
                        'Solar energy': 'solarenergy',
                        'Relative humidity':'humidity'})
X = df.drop(columns=['Site Performance Estimate'], axis=1)
y = df['Site Performance Estimate']

# Splitting the dfset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing the df
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Read data from API
vs_test = pd.read_csv("Archive/sample_visual_crossing_df.csv")

# Preprocess the data
vs_test['datetime'] = pd.to_datetime(vs_test['datetime'])
vs_test['year'] = vs_test['datetime'].dt.year
vs_test['month'] = vs_test['datetime'].dt.month
vs_test['day'] = vs_test['datetime'].dt.day
vs_test = vs_test.drop(columns=['datetime', 'name'])

# Reorder column and MinMaxScaler
vs_test = vs_test[['cloudcover', 'visibility', 'temp', 'dew', 'humidity', 'windspeed',
       'solarenergy', 'year', 'month', 'day']]
vs_test = scaler.transform(vs_test)

In [7]:
# Predict using AdaBoosting
final_ada = AdaBoostRegressor(base_estimator=None, 
                              learning_rate=0.5, 
                              n_estimators=150,
                              random_state=42)
result = final_ada.fit(X_train, y_train)

# Final prediction
vs_pred = result.predict(vs_test)

In [8]:
vs_output = pd.read_csv("Archive/sample_visual_crossing_df.csv")
vs_output['predicted_solar'] = vs_pred
vs_output.to_csv("sample_output.csv")